# Benchmarking

# Using QPCA class

In [1]:
from QPCA.decomposition.Qpca import QPCA
import numpy as np
import pandas as pd
from QPCA.preprocessingUtilities.preprocessing import generate_matrix
import time
from qiskit_ibm_runtime import QiskitRuntimeService, Sampler, Options
from qiskit_aer import AerSimulator, Aer
from qiskit.primitives import BackendSampler

In [2]:
df_=pd.DataFrame(columns=['matrix_idx','seed','n_shots','eigenvalues','reconstructed_eigenvalues','1st_original_eigenvector',
                     '1st_reconstructed_eigenvector','2nd_original_eigenvector','2nd_reconstructed_eigenvector',
                    '3th_original_eigenvector','3th_reconstructed_eigenvector','4th_original_eigenvector',
                     '4th_reconstructed_eigenvector','l2_error_first_eig','l2_error_second_eig','l2_error_third_eig',
                     'l2_error_fourth_eig','delta_error','runtime'])
df_

,matrix_idx,seed,n_shots,eigenvalues,reconstructed_eigenvalues,1st_original_eigenvector,1st_reconstructed_eigenvector,2nd_original_eigenvector,2nd_reconstructed_eigenvector,3th_original_eigenvector,3th_reconstructed_eigenvector,4th_original_eigenvector,4th_reconstructed_eigenvector,l2_error_first_eig,l2_error_second_eig,l2_error_third_eig,l2_error_fourth_eig,delta_error,runtime


In [3]:
def find_nearest(array, value):
    array = np.asarray(array)
    idx = (np.abs(array - value)).argmin()
    min_=(np.abs(array - value)).min()
    return array[idx],min_

In [4]:
#seed per matrici 2x2 1235,1235,19,87,16,856,100,4444,107,500
# seed per matrici 4x4 1235,1235,543,4747,6734,543,1000,1500,900,3500

real_data=False

noisy_experiments=False

real_hw=False

matrix_dimension=2

resolutions=[8]

shots_numbers=[100,500,1500,10000,100000,500000,1000000]

if matrix_dimension==2:
    seed=[1235,1235,19,87,16,856,100,4444,107,500]
elif matrix_dimension==4:
    seed=[1235,1235,543,4747,6734,543,1000,1500,900,3500]
    
if real_hw == True or noisy_experiments == True:
    service = QiskitRuntimeService(channel="ibm_cloud", token="", instance="")
    old_backend = service.get_backend('ibm_algiers')
    if noisy_experiments == True:
        backend = BackendSampler(AerSimulator.from_backend(old_backend)) 
        backend.set_transpile_options(seed_transpiler=42)
        # backend.set_options(shots=n_shots)
    else:     
        options = Options(transpilation={'seed_transpiler':42})
        backend = Sampler(old_backend, options=options)

else:
    backend = BackendSampler(Aer.get_backend('qasm_simulator'))
    backend.set_transpile_options(seed_transpiler=42)
    # backend.set_options(shots=n_shots)

if real_data==True:
    df = pd.read_csv("Dataset/covariance_matrix")

In [ ]:
for e in range(10):
    
    if real_data==False:
        
        if e==0:
            re_paper=True
        else:
            re_paper=False
        input_matrix=generate_matrix(matrix_dimension=matrix_dimension,replicate_paper=re_paper,seed=seed[e])#,eigenvalues_list=eigenvalues_list[e])
    else:
        
        jj=e+(matrix_dimension-1)
        input_matrix=df.iloc[e:jj+1,e:jj+1].to_numpy()
    
    for resolution in resolutions:
        start_time_fit=time.time()
        qpca=QPCA().fit(input_matrix,resolution=resolution,optimized_qram=True)
        end_time_fit=time.time()
        total_time_fit=end_time_fit-start_time_fit
        print(qpca.input_matrix)
        print(np.linalg.eig(qpca.input_matrix))
        
        eigenValues,eigenVectors=np.linalg.eig(qpca.input_matrix)
        idx = eigenValues.argsort()[::-1]   
        original_eigenValues = eigenValues[idx]
        original_eigenVectors = eigenVectors[:,idx]
        basic_dict={original_eigenValues[o]:o for o in range(len(original_eigenValues))}

        error_list=[]
        delta_list=[]
        shots_dict={}

        print('\n')
        print('\033[1m'+'Resolution: ',resolution)
        print('\n')
        for s in shots_numbers:
            dict_={}
            number_of_errors=np.zeros(len(input_matrix))
            number_of_errors[:]=np.nan

            reconstructed_ev=np.empty((len(input_matrix),len(input_matrix)))
            reconstructed_ev[:] = np.nan

            start_time_reconstruction=time.time()
            eigenvalues,eigenvectors=qpca.eigenvectors_reconstruction(n_shots=s,n_repetitions=1,backend=backend)
            end_time_reconstruction=time.time()
            total_time_reconstruction=end_time_reconstruction-start_time_reconstruction

            total_runtime=total_time_fit+total_time_reconstruction
        
            print('total runtime: ',total_runtime)
            
            k = eigenvalues.argsort()[::-1]   
            eigenvalues = eigenvalues[k]
            eigenvectors = eigenvectors[:,k]
            
            eigenvectors=np.array([eigenvectors[:,i] for i in range(len(eigenvalues))])
            
            print('reconstructed_eigenvalues:', eigenvalues)
            print('reconstructed_eigenvectors:', eigenvectors)
            
            eigenvalues_to_keep=[]
            try:
                
                results=qpca.spectral_benchmarking(eigenvector_benchmarking=True,sign_benchmarking=False ,eigenvalues_benchmarching=False,print_distances=True,only_first_eigenvectors=False,
                                                                plot_delta=True,distance_type='l2',error_with_sign=True,hide_plot=False,print_error=True)

                eig_to_keep_from_results=[]
                for e1 in results[0][0]:

                    x1,_=find_nearest(original_eigenValues,e1[0])  
                    k1=basic_dict[x1]
                    number_of_errors[k1]=e1[1]
                    eig_to_keep_from_results.append(e1[0])

                for e2, eig in enumerate(eigenvalues):
                    if eig in eig_to_keep_from_results:
                        x_1,_=find_nearest(original_eigenValues,eig) 
                        k_1=basic_dict[x_1]
                        eigenvalues_to_keep.append(eig)
                        reconstructed_ev[k_1]=eigenvectors[e2]

                print('held eigenvalues after benchmark cleaning: ', eigenvalues_to_keep)
                print('corresponding_eigenvectors: ',reconstructed_ev)
                
            
            except: #se il benchmark taglia via tutti gli autovalori trovati perchè non buoni, andiamo in except altrimenti si rompe
                results=np.nan
                if matrix_dimension==4:
                    dict_.update({'matrix_idx':int(e),'dimension':matrix_dimension,'n_shots':int(s),'eigenvalues':original_eigenValues*qpca.input_matrix_trace,'reconstructed_eigenvalues':sorted(np.array(eigenvalues_to_keep)*qpca.input_matrix_trace,reverse=True),'1st_original_eigenvector':original_eigenVectors[:,0],
                             '1st_reconstructed_eigenvector':reconstructed_ev[0],'2nd_original_eigenvector':original_eigenVectors[:,1],'2nd_reconstructed_eigenvector':reconstructed_ev[1],
                            '3th_original_eigenvector':original_eigenVectors[:,2],'3th_reconstructed_eigenvector':reconstructed_ev[2],'4th_original_eigenvector':original_eigenVectors[:,3],
                             '4th_reconstructed_eigenvector':reconstructed_ev[3],'l2_error_first_eig':number_of_errors[0],'l2_error_second_eig':number_of_errors[1],'l2_error_third_eig':number_of_errors[2],
                             'l2_error_fourth_eig':number_of_errors[3],'delta_error':results,'runtime':total_runtime})
                    
                elif matrix_dimension==2:
                    
                    dict_.update({'matrix_idx':int(e),'dimension':matrix_dimension,'resolution':int(resolution),'eigenvalues':original_eigenValues*qpca.input_matrix_trace,'reconstructed_eigenvalues':sorted(np.array(eigenvalues_to_keep)*qpca.input_matrix_trace,reverse=True),'1st_original_eigenvector':original_eigenVectors[:,0],
                         '1st_reconstructed_eigenvector':reconstructed_ev[0],'2nd_original_eigenvector':original_eigenVectors[:,1],'2nd_reconstructed_eigenvector':reconstructed_ev[1],
                              'l2_error_first_eig':number_of_errors[0],'l2_error_second_eig':number_of_errors[1],'delta_error':results,'runtime':total_runtime})
                    
                    
                df_=df_.append(dict_,ignore_index=True)
                print('\n')
                print('\n')
                continue
                
  
            
            if len(input_matrix)==4:
       
                dict_.update({'matrix_idx':int(e),'dimension':matrix_dimension,'n_shots':int(s),'eigenvalues':original_eigenValues*qpca.input_matrix_trace,'reconstructed_eigenvalues':sorted(np.array(eigenvalues_to_keep)*qpca.input_matrix_trace,reverse=True),'1st_original_eigenvector':original_eigenVectors[:,0],
                         '1st_reconstructed_eigenvector':reconstructed_ev[0],'2nd_original_eigenvector':original_eigenVectors[:,1],'2nd_reconstructed_eigenvector':reconstructed_ev[1],
                        '3th_original_eigenvector':original_eigenVectors[:,2],'3th_reconstructed_eigenvector':reconstructed_ev[2],'4th_original_eigenvector':original_eigenVectors[:,3],
                         '4th_reconstructed_eigenvector':reconstructed_ev[3],'l2_error_first_eig':number_of_errors[0],'l2_error_second_eig':number_of_errors[1],'l2_error_third_eig':number_of_errors[2],
                         'l2_error_fourth_eig':number_of_errors[3],'delta_error':results[0][1],'runtime':total_runtime})
                
            elif len(input_matrix)==3:
       
                dict_.update({'matrix_idx':int(e),'dimension':matrix_dimension,'n_shots':int(s),'eigenvalues':original_eigenValues*qpca.input_matrix_trace,'reconstructed_eigenvalues':sorted(np.array(eigenvalues_to_keep)*qpca.input_matrix_trace,reverse=True),'1st_original_eigenvector':original_eigenVectors[:,0],
                         '1st_reconstructed_eigenvector':reconstructed_ev[0],'2nd_original_eigenvector':original_eigenVectors[:,1],'2nd_reconstructed_eigenvector':reconstructed_ev[1],
                        '3th_original_eigenvector':original_eigenVectors[:,2],'3th_reconstructed_eigenvector':reconstructed_ev[2],'l2_error_first_eig':number_of_errors[0],'l2_error_second_eig':number_of_errors[1],'l2_error_third_eig':number_of_errors[2],'delta_error':results[0][1],'runtime':total_runtime})
            else:
                dict_.update({'matrix_idx':int(e),'dimension':matrix_dimension,'n_shots':int(s),'eigenvalues':original_eigenValues*qpca.input_matrix_trace,'reconstructed_eigenvalues':sorted(np.array(eigenvalues_to_keep)*qpca.input_matrix_trace,reverse=True),'1st_original_eigenvector':original_eigenVectors[:,0],
                         '1st_reconstructed_eigenvector':reconstructed_ev[0],'2nd_original_eigenvector':original_eigenVectors[:,1],'2nd_reconstructed_eigenvector':reconstructed_ev[1],
                              'l2_error_first_eig':number_of_errors[0],'l2_error_second_eig':number_of_errors[1],'delta_error':results[0][1],'runtime':total_runtime})

            df_=df_.append(dict_,ignore_index=True)

In [ ]:
df_

In [ ]:
df_.to_csv('benchmark/real_experiments/real_data_benchmark_3x3_8resolution_threshold01.csv',index=False)